In [8]:
import spacy 

def extraire_mots_cles(requete):
    # modèle de langue en français
    nlp = spacy.load("fr_core_news_sm")
    
    requete_utilisateur = requete.lower()

    # traiter la requête avec spaCy
    doc = nlp(requete_utilisateur)
    
    # Extraire les mots-clés pertinents
    mots_cles = []
    for token in doc:
        # condition pour vérifier si le token est un nom ou un adjectif
        if (token.pos_ == "NOUN" and token.dep_ not in ["ROOT"]) or (token.pos_ == "ADV" and token.dep_ != "advmod") or (token.pos_ == "PRON" and token.dep_ != "dep") or (token.dep_ == "ROOT" and token.pos_ not in ["NOUN","VERB"]) or token.pos_ == "ADJ" or (token.pos_ == "PROPN" and token.dep_ in ["nmod","punct"]) :
            #condition supplémentaire pour exclure les mots vides et les ponctuations
            if not token.is_stop and not token.is_punct:
                mots_cles.append(token.text)
                
    # les mots-clés extraits
    if mots_cles:
        print("Mots-clés extraits :", mots_cles)
    else:
        print("Aucun mot-clé pertinent extrait.")

# demander à l'utilisateur d'entrer une requête
requete = input("Entrez votre requête : ")

# Appeler la fonction pour extraire les mots-clés
extraire_mots_cles(requete)

Entrez votre requête : je cherches des articles sur le cancer
Mots-clés extraits : ['articles', 'cancer']


In [220]:
nlp = spacy.load("fr_core_news_sm")
doc = nlp("je cherche des article sur le sars cov-2")
doc2  = nlp("cherche-moi un article sur la covid-19 pour les personnes âgées de plus de 70ans et de big data")


doc3 = nlp("cherche-moi un article sur la covid-19 pour les personnes âgées de plus de 70 ans et de bigdata")
#cherche-moi un article sur la covid-19 pour les personnes âgées de plus de 70 ans et de bigdata


for tok in doc:
  print(tok.text, "...", tok.pos_, ".......", tok.dep_)

print("====================================================")
for tok in doc2:
  print(tok.text, "...", tok.pos_, ".......", tok.dep_)

print("====================================================")
for tok in doc3:
  print(tok.text, "...", tok.pos_, ".......", tok.dep_)

je ... PRON ....... nsubj
cherche ... VERB ....... ROOT
des ... ADP ....... det
article ... NOUN ....... obj
sur ... ADP ....... case
le ... DET ....... det
sars ... NOUN ....... nmod
cov-2 ... PROPN ....... punct
cherche ... NOUN ....... ROOT
-moi ... PRON ....... dep
un ... DET ....... det
article ... NOUN ....... nmod
sur ... ADP ....... case
la ... DET ....... det
covid-19 ... NOUN ....... nmod
pour ... ADP ....... case
les ... DET ....... det
personnes ... NOUN ....... nmod
âgées ... ADJ ....... amod
de ... ADP ....... case
plus ... ADV ....... advmod
de ... ADP ....... fixed
70ans ... PROPN ....... obl:arg
et ... CCONJ ....... cc
de ... ADP ....... case
big ... PRON ....... nsubj
data ... ADV ....... conj
cherche ... NOUN ....... ROOT
-moi ... PRON ....... dep
un ... DET ....... det
article ... NOUN ....... nmod
sur ... ADP ....... case
la ... DET ....... det
covid-19 ... NOUN ....... nmod
pour ... ADP ....... case
les ... DET ....... det
personnes ... NOUN ....... nmod
âgées ...

In [2]:
from bs4 import BeautifulSoup
import requests
import urllib.request, urllib.parse, urllib.error
import re
import ssl
import json
import calendar
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy 


def extraire_mots_cles(requete):

    nlp = spacy.load("fr_core_news_sm")
    
    requete_utilisateur = requete.lower()

    doc = nlp(requete_utilisateur)

    mots_cles = []
    for token in doc:

        if (token.pos_ == "NOUN" and token.dep_ not in ["ROOT"]) or \
           (token.pos_ == "ADV" and token.dep_ != "advmod") or \
           (token.pos_ == "PRON" and token.dep_ != "dep") or \
           (token.dep_ == "ROOT" and token.pos_ not in ["NOUN","VERB"]) or \
           token.pos_ == "ADJ" or \
           (token.pos_ == "PROPN" and token.dep_ in ["nmod","punct"]):

            if not token.is_stop and not token.is_punct:
                mots_cles.append(token.text)
    

    mots_cles = [mot for mot in mots_cles if mot.lower() not in ['article','articles']]
    mots_cles_format = ' '.join(mots_cles)
    
    return mots_cles_format

def format_mot(motcle):
    if ' ' in motcle:
        format_motcle = motcle.replace(' ', '+')
    else:
        format_motcle = motcle
    return format_motcle

def strip_brackets(s): 
    no_brackets = "" 
    dont_want = ['[', ']']
    for char in s: 
        if char not in dont_want:
            no_brackets += char
    return no_brackets 

def get_bibliography(soup):
    article = soup.find('article')
    journal = soup.find('journal')
    listauteur = article.find('authorlist')
    auteur = ""
    if listauteur:
        initials = listauteur.find_all('initials')
        lastnames = listauteur.find_all('lastname')
        for i in range(len(lastnames)):
            if i < len(initials):
                initial = initials[i].text
                auteur += initial
                auteur += '. '
            else:
                auteur += ' '
            nom = lastnames[i].text
            auteur += nom
            if i == len(lastnames) - 2:
                auteur += ' and '
            elif i != len(lastnames) - 1:
                auteur += ', '
        auteur += ", "

    titreArticle = ''
    if article.find('articletitle'):
            titreArticle = '"'
            titre_str = article.find('articletitle').text
            titre_str = strip_brackets(titre_str)
            titreArticle  += titre_str
            if titreArticle [-1] == '.':
                titreArticle  += ''
            else:
                titreArticle  += '.'           
    titreJournal = ''
    if journal.find('title'):
        titreJournal = journal.find('title').text
        titreJournal += ' '
     
    JournalIssue = journal.find('journalissue')
    month = JournalIssue.find('month')
    date = ''
    if month:
        month = JournalIssue.find('month').text
        if len(month)<3:
            month_int = int(str(month))
            month = calendar.month_abbr[month_int]
        year = JournalIssue.find('year').text
        date = '('
        date += month
        date += '. '
        date += year
        date += '). '
    elif JournalIssue.find('year'):
        date = '('
        date += JournalIssue.find('year').text
        date += '). '      
    else: ''

    if soup.find('articleid'):
        doi_pii = article.find_all('elocationid')
        doi_pii_str = ""
        if len(doi_pii)>1:
            if 'doi' in str(doi_pii[0]):
                doi_pii = doi_pii[0].text
                doi_pii_str += "DOI "
                doi_pii_str += doi_pii
                doi_pii_str += "."
            elif 'doi' in str(doi_pii[1]):
                doi_pii = doi_pii[1].text
                doi_pii_str += "DOI "
                doi_pii_str += doi_pii
                doi_pii_str += "."
        elif len(doi_pii) == 1:
            if 'doi' in str(doi_pii[0]):
                doi_pii = doi_pii[0].text
                doi_pii_str += "DOI "
                doi_pii_str += doi_pii
                doi_pii_str += "."
            elif 'pii' in str(doi_pii[0]):
                doi_pii = doi_pii[0].text
                doi_pii_str += "PII "
                doi_pii_str += doi_pii
                doi_pii_str += "."
    
    resume = ''
    if article.find('abstracttext'):
        resume = article.find('abstracttext').text
    
    result = []
    result.append(auteur)
    result.append(titreArticle)
    result.append(titreJournal)
    result.append(date)
    result.append(doi_pii_str)
    result.append(resume)

    return result

def get_pubmed_ids(url, num):
    pageweb = urllib.request.urlopen(url).read()
    dict_page = json.loads(pageweb)
    return dict_page["esearchresult"]["idlist"]

def get_article_data(pubmed_ids):
    articles_data = []
    for link in pubmed_ids:
        url = "http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&retmode=xml&id=idlist"
        url = url.replace('idlist', link)
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")
        article = get_bibliography(soup)
        lien = f"https://www.ncbi.nlm.nih.gov/pubmed/{link}"
        article.append(lien) 
        articles_data.append(article)
    return articles_data

def calculate_similarity(keyword, article_titles):
    keyword_vector = [keyword]
    corpus = keyword_vector + article_titles
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)
    keyword_tfidf_vector = tfidf_matrix[0]
    article_titles_tfidf_matrix = tfidf_matrix[1:]
    cosine_similarities = cosine_similarity(keyword_tfidf_vector, article_titles_tfidf_matrix)
    return cosine_similarities.flatten()

In [2]:
def main():
    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&retmode=json&retmax=NUM&sort=relevance&term=KEYWORD"
    mot_cle = str(input('Veuillez saisir votre requête : '))
    motcle  = extraire_mots_cles(mot_cle)
    num = int(input('Nombre de résultats souhaitez : '))
    url = url.replace('NUM', str(num))
    url = url.replace('KEYWORD', format_mot(motcle))

    try:
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        pass
    else:
        ssl._create_default_https_context = _create_unverified_https_context
    
    pubmed_ids = get_pubmed_ids(url, num)
    articles_data = get_article_data(pubmed_ids)
    df = pd.DataFrame(articles_data)
    df.columns = ['Auteur', 'Nom_Article', 'Journal', 'Date', 'DOI', 'Résume', 'lien']
    
    
    similarity_scores = calculate_similarity(motcle, df['Nom_Article'].tolist())
    df['Similarité_Cosinus'] = similarity_scores

    file_name = motcle.replace(' ', '_') + '_' + str(num) + '.csv'
    df.to_csv(file_name)

    print("Fichier CSV enregistré avec succès.")

In [5]:
if __name__ == "__main__":
    main()

Veuillez saisir votre requête : je cherches des articles sur le cancer 
Nombre de résultats souhaitez : 20
Fichier CSV enregistré avec succès.


In [3]:
df = pd.read_csv("cancer_20.csv")
df.head()

,Unnamed: 0,Auteur,Nom_Article,Journal,Date,DOI,Résume,lien,Similarité_Cosinus
0,0,"LA. Torre, RL. Siegel, EM. Ward and A. Jemal,","""Global Cancer Incidence and Mortality Rates a...","Cancer epidemiology, biomarkers & prevention :...",(Jan. 2016).,DOI 10.1158/1055-9965.EPI-15-0578.,There are limited published data on recent can...,https://www.ncbi.nlm.nih.gov/pubmed/26667886,0.103376
1,1,"E. Judasz, N. Lisiak, P. Kopczyński, M. Taube ...","""The Role of Telomerase in Breast Cancer's Res...",International journal of molecular sciences,(Oct. 2022).,DOI 10.3390/ijms232112844.,"Currently, breast cancer appears to be the mos...",https://www.ncbi.nlm.nih.gov/pubmed/36361634,0.117683
2,2,"PS. Roy and BJ. Saikia,","""Cancer and cure: A critical analysis.",Indian journal of cancer,(2016).,DOI 10.4103/0019-509X.200658.,Cancer is one of the most dreaded diseases of ...,https://www.ncbi.nlm.nih.gov/pubmed/28244479,0.165221
3,3,"TA. Graham and A. Sottoriva,","""Measuring cancer evolution from the genome.",The Journal of pathology,(Jan. 2017).,DOI 10.1002/path.4821.,The temporal dynamics of cancer evolution rema...,https://www.ncbi.nlm.nih.gov/pubmed/27741350,0.141594
4,4,"E. Gadaleta, GJ. Thorn, H. Ross-Adams, LJ. Jon...","""Field cancerization in breast cancer.",The Journal of pathology,(Jul. 2022).,DOI 10.1002/path.5902.,Breast cancer affects one in seven women world...,https://www.ncbi.nlm.nih.gov/pubmed/35362092,0.167946


In [4]:
df['Similarité_Cosinus']

0     0.103376
1     0.117683
2     0.165221
3     0.141594
4     0.167946
5     0.109817
6     0.122584
7     0.203737
8     0.182636
9     0.215664
10    0.167513
11    0.167513
12    0.105512
13    0.203737
14    0.096162
15    0.241492
16    0.100978
17    0.108498
18    0.207586
19    0.101800
Name: Similarité_Cosinus, dtype: float64

In [7]:
if __name__ == "__main__":
    main()

Veuillez saisir votre requête : je veut des article sur le covid-19
Nombre de résultats souhaitez : 15


/Users/berkaniyacine/anaconda3/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Fichier CSV enregistré avec succès.


In [5]:
df = pd.read_csv("covid-19_15.csv")
df.head()

,Unnamed: 0,Auteur,Nom_Article,Journal,Date,DOI,Résume,lien,Similarité_Cosinus
0,0,"J. Majumder and T. Minko,","""Recent Developments on Therapeutic and Diagno...",The AAPS journal,(Jan. 2021).,DOI 10.1208/s12248-020-00532-2.,The ongoing pandemic of coronavirus disease 20...,https://www.ncbi.nlm.nih.gov/pubmed/33400058,0.188332
1,1,"M. Yüce, E. Filiztekin and KG. Özkaya,","""COVID-19 diagnosis -A review of current methods.",Biosensors & bioelectronics,(Jan. 2021).,DOI 10.1016/j.bios.2020.112752.,A fast and accurate self-testing tool for COVI...,https://www.ncbi.nlm.nih.gov/pubmed/33126180,0.250644
2,2,"K. Habas, C. Nganwuchu, F. Shahzad, R. Gopalan...","""Resolution of coronavirus disease 2019 (COVID...",Expert review of anti-infective therapy,(Dec. 2020).,DOI 10.1080/14787210.2020.1797487.,Coronavirus disease 2019 (COVID-19) was first ...,https://www.ncbi.nlm.nih.gov/pubmed/32749914,0.238548
3,3,"R. Ochani, A. Asad, F. Yasmin, S. Shaikh, H. K...","""COVID-19 pandemic: from origins to outcomes. ...",Le infezioni in medicina,(Mar. 2021).,NaN,Severe Acute Respiratory Syndrome Coronavirus-...,https://www.ncbi.nlm.nih.gov/pubmed/33664170,0.123061
4,4,"Q. Fernandes, VP. Inchakalody, M. Merhi, S. Me...","""Emerging COVID-19 variants and their impact o...",Annals of medicine,(Dec. 2022).,DOI 10.1080/07853890.2022.2031274.,The emergence of novel and evolving variants o...,https://www.ncbi.nlm.nih.gov/pubmed/35132910,0.143493


In [6]:
df['Similarité_Cosinus']

0     0.188332
1     0.250644
2     0.238548
3     0.123061
4     0.143493
5     0.186494
6     0.227404
7     0.197449
8     0.147839
9     0.226199
10    0.188701
11    0.202593
12    0.176860
13    0.195490
14    0.169613
Name: Similarité_Cosinus, dtype: float64

In [10]:
if __name__ == "__main__":
    main()

Veuillez saisir votre requête : je veux des articles sur le SARS CoV-2
Nombre de résultats souhaitez : 12


/Users/berkaniyacine/anaconda3/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Fichier CSV enregistré avec succès.


In [7]:
df = pd.read_csv("sars_cov-2_12.csv")
df.head()

,Unnamed: 0,Auteur,Nom_Article,Journal,Date,DOI,Résume,lien,Similarité_Cosinus
0,0,"CB. Jackson, M. Farzan, B. Chen and H. Choe,","""Mechanisms of SARS-CoV-2 entry into cells.",Nature reviews. Molecular cell biology,(Jan. 2022).,DOI 10.1038/s41580-021-00418-x.,The unprecedented public health and economic i...,https://www.ncbi.nlm.nih.gov/pubmed/34611326,0.226033
1,1,"P. V'kovski, A. Kratzel, S. Steiner, H. Stalde...","""Coronavirus biology and replication: implicat...",Nature reviews. Microbiology,(Mar. 2021).,DOI 10.1038/s41579-020-00468-6.,The SARS-CoV-2 pandemic and its unprecedented ...,https://www.ncbi.nlm.nih.gov/pubmed/33116300,0.243756
2,2,"C. Bai, Q. Zhong and GF. Gao,","""Overview of SARS-CoV-2 genome-encoded proteins.",Science China. Life sciences,(Feb. 2022).,DOI 10.1007/s11427-021-1964-4.,Severe acute respiratory syndrome Coronavirus ...,https://www.ncbi.nlm.nih.gov/pubmed/34387838,0.232743
3,3,"H. Yang and Z. Rao,","""Structural biology of SARS-CoV-2 and implicat...",Nature reviews. Microbiology,(Nov. 2021).,DOI 10.1038/s41579-021-00630-8.,"The COVID-19 pandemic, caused by severe acute ...",https://www.ncbi.nlm.nih.gov/pubmed/34535791,0.201578
4,4,"N. Kirtipal, S. Bharadwaj and SG. Kang,","""From SARS to SARS-CoV-2, insights on structur...","Infection, genetics and evolution : journal of...",(Nov. 2020).,DOI 10.1016/j.meegid.2020.104502.,"Human Coronaviruses (HCoV), periodically emerg...",https://www.ncbi.nlm.nih.gov/pubmed/32798769,0.213575


In [8]:
df['Similarité_Cosinus']

0     0.226033
1     0.243756
2     0.232743
3     0.201578
4     0.213575
5     0.155079
6     0.223255
7     0.204499
8     0.119247
9     0.302966
10    0.184888
11    0.144745
Name: Similarité_Cosinus, dtype: float64